In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
diamonds = pd.read_csv('./data/diamondsql.csv')

In [4]:
diamonds = diamonds.drop(['index_id', 'city_id', 'city_id.1','index_id.1','cut_id', 'color_id', 'clarity_id', 'cut_id.1', 'clarity_id.1', 'color_id.1', 'index_id.2'], axis = 1)

In [5]:
diamonds_encoded = pd.get_dummies(diamonds, columns=['city', 'cut', 'clarity', 'color'])

In [6]:
X = diamonds_encoded.drop('price', axis=1)
y = diamonds_encoded['price']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (32364, 39), X_test: (8091, 39), y_train: (32364,), y_test: (8091,)


In [8]:

# Calcular la matriz de correlación
correlation_matrix = diamonds_encoded.corr()

# Seleccionar la variable objetivo (en este ejemplo, 'price') y ordenar las correlaciones en orden descendente
target_correlation = correlation_matrix['price'].sort_values(ascending=False)

# Definir un umbral de correlación por encima del cual consideraremos que las características están altamente correlacionadas con la variable objetivo
correlation_threshold = 0.5

# Filtrar las características que tienen una correlación por encima del umbral
selected_features = target_correlation[abs(target_correlation) > correlation_threshold].index.tolist()

# Eliminar la variable objetivo de la lista de características seleccionadas
selected_features.remove('price')

# Seleccionar solo las características que pasan el umbral de correlación
df_selected = diamonds_encoded[selected_features]


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

RMSE: 563.917903331059


/Users/guillemorato/miniconda3/envs/sklearn_/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
diamonds_test = pd.read_csv('./data/diamonds_test.csv')

In [11]:
diamonds_encoded_test = pd.get_dummies(diamonds_test, columns=['city', 'cut', 'clarity', 'color'])

In [13]:
X_prueba = diamonds_encoded_test.drop('id', axis=1)
X_prueba_scaled = scaler.transform(X_prueba)
y_prueba_pred = model.predict(X_prueba_scaled)


In [15]:
solution = []
for i, value in enumerate(y_prueba_pred):
    solution.append((i,value))
    

In [16]:
solution_df = pd.DataFrame(solution)


In [17]:
solution_df.columns = ['id', 'price']

In [18]:
solution_df['price'].to_csv('StandarScaler.csv')